In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import numpy as np
from astropy.io import ascii
from scipy.integrate import quad
import sys
sys.path.append("/Users/annaho/Dropbox/Projects/Research/ZTF18abukavn/code")
from load_lc import get_lc
import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import fmin
from scipy.optimize import least_squares
import matplotlib.gridspec as gridspec
from astropy.cosmology import Planck15
import json
import math
plt.rc('text', usetex=True)
plt.rc('font',family='helvetica')

In [4]:
import emcee
import corner

In [7]:
# get the single-band light curves

dt, filt, mag, emag = get_lc()

In [9]:
kappa=0.2
Msol=2*1e33
sigmaS=5.67*1e-5
D_L=Planck15.luminosity_distance(z=0.03154).cgs.value
explEnergy=1E52
Mcore=0.23*Msol

In [10]:
def planckBB(x,T,R,D):
    h=6.62e-27
    c=3e10
    kB=1.38e-16
    return np.pi*2*h*c**2/x**5*1/(np.exp(h*c/(x*kB*T))-1)*(R/D)**2/1e8

In [11]:
def bandFlux(filterFile,T,R,D,redshift):
    waves=np.array(filterFile['col1'])/1e8/(1+redshift)
    transm=np.array(filterFile['col2'])

    fluxes=planckBB(waves,T,R,D)
    effFlux=np.sum(fluxes*transm)/np.sum(transm)
    return effFlux

In [12]:
def getBandMag(filterFile,T,R,D,redshift,effWave):
    tempBandFlux=bandFlux(filterFile,T,R,D,redshift)
    tempBandFluxNu=3.34*10**4*(effWave)**2*tempBandFlux
    if tempBandFluxNu<=0:
        return 99
    return -5/2*np.log10(tempBandFluxNu)+8.90

In [13]:
def convertMagToFlux(magArray,magErrArray,effWave):
    bandFluxLam=np.zeros(len(magArray))
    bandFluxLamErr=np.zeros(len(magArray))
    for i in range(len(magArray)):
        bandFluxNu=10**(-2/5*(magArray[i]-8.90))
        bandFluxLam[i]=bandFluxNu/(3.34*10**4*effWave**2)
        bandFluxLamErr[i]=magErrArray[i]*bandFluxLam[i]/1.08
        
    return bandFluxLam, bandFluxLamErr

In [14]:
def shockModel(t,Me,Re):
    lum=[]
    R=[]
    temp=[]
    for i in range(len(t)):
        if t[i]<0:
            lum.append(0)
            R.append(0)
            temp.append(0)
            
        else:
            Mc=Mcore
            E=explEnergy
            ve=1.5*10**9*(E/1e51)**0.5*(Mc/(3*Msol))**(-0.35)*(Me/(0.01*Msol))**(-0.15)
            tp=0.9*(kappa/0.34)**0.5*(E/1e51)**(-1/4)*(Mc/Msol)**0.17*(Me/(0.01*Msol))**0.57 * 86400
            Ee=1/2*Me*ve**2
    
            te=Re/ve
            lum.append(te*Ee/tp**2*np.exp(-t[i]*(t[i]+2*te)/(2*tp**2)))
            #lum.append(8.27*1e42*(kappa/0.34)**(-1)*(ve/1e9)**2*(Re/1e13)*(Mc/Msol)**0.01*np.exp(-4.135*10**(-11)*t[i]*(t[i]*(ve/1e9) + 2*10**4*(Re/1e13))*(kappa/0.34)**(-1)*(Mc/Msol)**0.01*(Me/(0.01*Msol))**(-1)))
            R.append(Re+ve*t[i])
            temp.append((lum[i]/(sigmaS*4*np.pi*R[i]**2))**(1/4))
    
    return lum, R, temp

In [19]:
def computeFluxChiSquare(Me,Re,t0):
    chiSquare=0
    
    timeCutOff = 1.15
    
    timeArray=detTimes[uw2BandDets[0][np.where(detTimes[uw2BandDets]<timeCutOff)]]*86400
    magArray=detMags[uw2BandDets[0][np.where(detTimes[uw2BandDets]<timeCutOff)]]
    magErrArray=detErrs[uw2BandDets[0][np.where(detTimes[uw2BandDets]<timeCutOff)]]
    effWave=2120
    filtObj=filterData[0]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    bandF=np.zeros(len(timeArray))
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    for i in range(len(timeArray)):
        if tempFit[i]!=0:
            bandF[i]=bandFlux(filtObj,tempFit[i],RFit[i],D_L,redshift)
        else:
            bandF[i]=99
        
        chiSquare=chiSquare+(magFlux[i]-bandF[i])**2/magFluxErr[i]**2
        #print(magArray[i],bandMag[i],(magArray[i]-bandMag[i])**2)
        
    timeArray=detTimes[uw1BandDets[0][np.where(detTimes[uw1BandDets]<timeCutOff)]]*86400
    magArray=detMags[uw1BandDets[0][np.where(detTimes[uw1BandDets]<timeCutOff)]]
    magErrArray=detErrs[uw1BandDets[0][np.where(detTimes[uw1BandDets]<timeCutOff)]]
    effWave=2910
    filtObj=filterData[1]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    bandF=np.zeros(len(timeArray))
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    for i in range(len(timeArray)):
        if tempFit[i]!=0:
            bandF[i]=bandFlux(filtObj,tempFit[i],RFit[i],D_L,redshift)
        else:
            bandF[i]=99
            
        chiSquare=chiSquare+(magFlux[i]-bandF[i])**2/magFluxErr[i]**2
        #print(magArray[i],bandMag[i],(magArray[i]-bandMag[i])**2)
     
    timeArray=detTimes[gBandDets[0][np.where(detTimes[gBandDets]<timeCutOff)]]*86400
    magArray=detMags[gBandDets[0][np.where(detTimes[gBandDets]<timeCutOff)]]
    magErrArray=detErrs[gBandDets[0][np.where(detTimes[gBandDets]<timeCutOff)]]
    #timeArray=np.append(timeArray,upTimes[gBandUps][-1]*86400)
    #magArray=np.append(magArray,upMags[gBandUps][-1])
    #magErrArray=np.append(magErrArray,magErrArray[0])
    
    effWave=4639
    filtObj=filterData[2]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    bandF=np.zeros(len(timeArray))
    
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    for i in range(len(timeArray)):
        if tempFit[i]!=0:
            bandF[i]=bandFlux(filtObj,tempFit[i],RFit[i],D_L,redshift)
        else:
            bandF[i]=99
            
        chiSquare=chiSquare+(magFlux[i]-bandF[i])**2/magFluxErr[i]**2
        #print(magArray[i],bandMag[i],(magArray[i]-bandMag[i])**2)
    
    timeArray=upTimes[gBandUps][-1]*86400
    magArray=upMags[gBandUps][-1]
    timeArray=np.append(timeArray,0)
    magArray=np.append(magArray,0)
    
    effWave=4639
    filtObj=filterData[2]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    if tempFit[0]!=0:
        bandFUse=bandFlux(filtObj,tempFit[0],RFit[0],D_L,redshift)
        if bandFUse>magFlux[0]:
            chiSquare=chiSquare+((magFlux[0]-bandFUse)/magFluxErr[0]*10)**2

    timeArray=detTimes[rBandDets[0][np.where(detTimes[rBandDets]<timeCutOff)]]*86400
    magArray=detMags[rBandDets[0][np.where(detTimes[rBandDets]<timeCutOff)]]
    magErrArray=detErrs[rBandDets[0][np.where(detTimes[rBandDets]<timeCutOff)]]
    effWave=6122
    filtObj=filterData[3]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    bandF=np.zeros(len(timeArray))
    
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    for i in range(len(timeArray)):
        if tempFit[i]!=0:
            bandF[i]=bandFlux(filtObj,tempFit[i],RFit[i],D_L,redshift)
        else:
            bandF[i]=99
            
        chiSquare=chiSquare+(magFlux[i]-bandF[i])**2/magFluxErr[i]**2
        #print(magArray[i],bandMag[i],(magArray[i]-bandMag[i])**2)
        
    timeArray=detTimes[iBandDets[0][np.where(detTimes[iBandDets]<timeCutOff)]]*86400
    magArray=detMags[iBandDets[0][np.where(detTimes[iBandDets]<timeCutOff)]]
    magErrArray=detErrs[iBandDets[0][np.where(detTimes[iBandDets]<timeCutOff)]]
    effWave=7439
    filtObj=filterData[4]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    bandF=np.zeros(len(timeArray))
    
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    for i in range(len(timeArray)):
        if tempFit[i]!=0:
            bandF[i]=bandFlux(filtObj,tempFit[i],RFit[i],D_L,redshift)
        else:
            bandF[i]=99
            
        chiSquare=chiSquare+(magFlux[i]-bandF[i])**2/magFluxErr[i]**2
        #print(magArray[i],bandMag[i],(magArray[i]-bandMag[i])**2)
        
    timeArray=detTimes[bBandDets[0][np.where(detTimes[bBandDets]<timeCutOff)]]*86400
    magArray=detMags[bBandDets[0][np.where(detTimes[bBandDets]<timeCutOff)]]
    magErrArray=detErrs[bBandDets[0][np.where(detTimes[bBandDets]<timeCutOff)]]
    effWave=4378
    filtObj=filterData[5]
    [lumFit,RFit,tempFit]=shockModel(timeArray-t0,Me,Re)
    bandF=np.zeros(len(timeArray))
    
    [magFlux,magFluxErr]=convertMagToFlux(magArray,magErrArray,effWave)
    
    for i in range(len(timeArray)):
        if tempFit[i]!=0:
            bandF[i]=bandFlux(filtObj,tempFit[i],RFit[i],D_L,redshift)
        else:
            bandF[i]=99
            
        chiSquare=chiSquare+(magFlux[i]-bandF[i])**2/magFluxErr[i]**2
        #print(magArray[i],bandMag[i],(magArray[i]-bandMag[i])**2)
            
    return chiSquare

In [20]:
Me_min=0.001*Msol
Me_max=0.02*Msol
R_min=5e12
R_max=8e13
t0_min=-1*86400
t0_max=-0.1*86400
#ve_min=2e8
#ve_max=1e9

In [21]:
filtFiles=['Swift_UVOT.UVW2.dat','Swift_UVOT.UVW1.dat','SLOAN_SDSS.g.dat','SLOAN_SDSS.r.dat','SLOAN_SDSS.i.dat','Generic_Johnson.B.dat']
filterData=[]
for i in range(len(filtFiles)):
    filterData.append(ascii.read(filtFiles[i]))

In [22]:
def lnlike(theta):
    Me, Re, t0 = theta
    return -computeFluxChiSquare(Me, Re, t0)

In [23]:
def lnprior(theta):
    Me, Re, t0 = theta
    if Me_min < Me < Me_max and R_min < Re < R_max and t0_min < t0 < t0_max:
        return 0.0
    return -np.inf

In [24]:
def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    
    return lp + lnlike(theta)

In [25]:
Me_chiMin = 3e31
Re_chiMin = 0.1e14
#ve_chiMin = 5e8
t0_chiMin = -5e+04

In [26]:
ndim, nwalkers = 3, 100
pos = [np.array([Me_chiMin, Re_chiMin, t0_chiMin]) + np.array([Me_chiMin, 0, 0])*1e-4*np.random.rand(1) + np.array([0, Re_chiMin, 0])*1e-4*np.random.rand(1) + np.array([0, 0, t0_chiMin])*1e-4*np.random.rand(1) for i in range(nwalkers)]


fig = corner.corner(pos, labels=["$M_e$", "$R_e$", "$t_0$"], plot_contours = False)
fig.savefig('cornerPrior_shock.pdf')
#fig.close()
plt.close()

In [27]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

In [28]:
sampler.run_mcmc(pos, 500)

emcee: Exception while calling your likelihood function:
  params: [ 3.00001581e+31  1.00005589e+13 -5.00044197e+04]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/annaho/anaconda/lib/python3.5/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-24-9c002454e316>", line 6, in lnprob
    return lp + lnlike(theta)
  File "<ipython-input-22-fd0a4cde3c89>", line 3, in lnlike
    return -computeFluxChiSquare(Me, Re, t0)
  File "<ipython-input-19-4bfc49656866>", line 6, in computeFluxChiSquare
    timeArray=detTimes[uw2BandDets[0][np.where(detTimes[uw2BandDets]<timeCutOff)]]*86400
NameError: name 'detTimes' is not defined


NameError: name 'detTimes' is not defined

In [26]:
samples = sampler.chain[:,100:,:].reshape((-1,ndim))

In [27]:
np.shape(samples)


(40000, 3)

In [28]:
samples[:,0] /= 2e31
samples[:,1] /= 1e14
#samples[:,2] /= 1e9
samples[:,2] /= 86400

In [29]:
fig = corner.corner(samples, labels=["$M_e$ (10$^{-2}$ M$_{\odot}$)", "$R_e$ (10$^{14}$ cm)", "$t_0$ (days)"], quantiles = [0.16, 0.5, 0.84], show_titles = True, labels_args={"fontsize": 30}, levels=(0.68,0.95), range = [[0.6,1.5],[0.19,0.42],[-0.75,-0.4]], verbose = True)
#truths=[Me_chiMin/2e31, Re_chiMin/1e14, t0_chiMin/86400]
fig.savefig("corner_shock.pdf")
#fig.close()
plt.close()

Quantiles:
[(0.16, 0.8172864816248235), (0.5, 0.8885252731383655), (0.84, 0.9802656099864695)]
Quantiles:
[(0.16, 0.27349926994767304), (0.5, 0.30095034675114407), (0.84, 0.3285330352094597)]
Quantiles:
[(0.16, -0.6245771407306792), (0.5, -0.5825381658755324), (0.84, -0.5418186526619159)]


In [726]:
kappa=0.2
Msol=2*1e33
sigmaS=5.67*1e-5
D_L=284.5*3.08*1e24
explEnergy=1.89e50
Mcore=0.31*Msol

In [727]:
sampler2 = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

In [728]:
sampler2.run_mcmc(pos, 500)

(array([[ 1.87930202e+31,  2.46380106e+13, -5.18506894e+04],
        [ 1.80213842e+31,  2.78806661e+13, -5.14191930e+04],
        [ 2.07834714e+31,  2.26394104e+13, -5.74089285e+04],
        [ 1.86610393e+31,  2.54712292e+13, -5.38423376e+04],
        [ 2.16781917e+31,  2.17665603e+13, -5.93326524e+04],
        [ 2.03778436e+31,  2.36573145e+13, -5.58108253e+04],
        [ 1.80676160e+31,  2.66009938e+13, -5.27645882e+04],
        [ 1.97773625e+31,  2.29831651e+13, -5.58797247e+04],
        [ 2.07677015e+31,  2.33660814e+13, -5.72910637e+04],
        [ 2.35284653e+31,  2.14125494e+13, -6.00474443e+04],
        [ 1.82882726e+31,  2.56703692e+13, -5.39553017e+04],
        [ 1.77138604e+31,  2.74130261e+13, -5.10172074e+04],
        [ 2.03633796e+31,  2.32355259e+13, -5.65859282e+04],
        [ 1.76186210e+31,  2.67720755e+13, -5.17928077e+04],
        [ 1.92782511e+31,  2.41257248e+13, -5.57040448e+04],
        [ 2.37614236e+31,  2.01230742e+13, -6.08699947e+04],
        [ 1.83036506e+31

In [729]:
samples2 = sampler2.chain[:,100:,:].reshape((-1,ndim))

In [730]:
samples2[:,0] /= 2e31
samples2[:,1] /= 1e14
#samples[:,2] /= 1e9
samples2[:,2] /= 86400

In [725]:
fig = corner.corner(samples2, labels=["$M_e$ (10$^{-2}$ M$_{\odot}$)", "$R_e$ (10$^{14}$ cm)", "$t_0$ (days)"], quantiles = [0.16, 0.5, 0.84], show_titles = True, labels_args={"fontsize": 30}, levels=(0.68,0.95), range = [[0.6,1.5],[0.19,0.42],[-0.75,-0.4]], verbose = True)
#truths=[Me_chiMin/2e31, Re_chiMin/1e14, t0_chiMin/86400]
fig.savefig("corner_shockLow.pdf")
#fig.close()
plt.close()

Quantiles:
[(0.16, 0.7559097718781828), (0.5, 0.8179044946853127), (0.84, 0.90089705107474)]
Quantiles:
[(0.16, 0.3242727486186948), (0.5, 0.3579391662527426), (0.84, 0.38915548454865)]
Quantiles:
[(0.16, -0.5741929478607761), (0.5, -0.5275463689897384), (0.84, -0.48481287448609817)]


In [731]:
fig = corner.corner(samples2, labels=["$M_e$ (10$^{-2}$ M$_{\odot}$)", "$R_e$ (10$^{14}$ cm)", "$t_0$ (days)"], quantiles = [0.16, 0.5, 0.84], show_titles = True, labels_args={"fontsize": 30}, levels=(0.68,0.95), range = [[0.6,1.5],[0.19,0.42],[-0.75,-0.4]], verbose = True)
#truths=[Me_chiMin/2e31, Re_chiMin/1e14, t0_chiMin/86400]
fig.savefig("corner_shockHigh.pdf")
#fig.close()
plt.close()

Quantiles:
[(0.16, 0.9128897455467931), (0.5, 0.9978987023551908), (0.84, 1.094974570945184)]
Quantiles:
[(0.16, 0.21658684461492397), (0.5, 0.2360298537483949), (0.84, 0.2581364982707984)]
Quantiles:
[(0.16, -0.6793100861166556), (0.5, -0.6432836791557692), (0.84, -0.6047462440337497)]
